# 🧠 AI Model Power Test - Group G9
## Testing the Medicinal Plant Detection Engine

This notebook demonstrates the **intelligence and accuracy** of our AI model by:
1. Loading the trained model
2. Testing on real medicinal plant images
3. Visualizing predictions with confidence scores
4. Analyzing model performance metrics

In [20]:
# Import required libraries
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import seaborn as sns

print(f"✅ TensorFlow Version: {tf.__version__}")
print(f"✅ GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

ModuleNotFoundError: No module named 'tensorflow'

## 📂 Load Model and Class Names

In [ ]:
# Paths
MODEL_PATH = Path("models/mobilenetv2_best.h5")
CLASS_NAMES_PATH = Path("models/class_names.json")
DATASET_PATH = Path("../dataset/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset")

# Load class names
with open(CLASS_NAMES_PATH, 'r') as f:
    class_indices = json.load(f)
    class_names = {v: k for k, v in class_indices.items()}

print(f"📊 Total Classes: {len(class_names)}")
print(f"\n🌿 Sample Classes:")
for i in range(min(10, len(class_names))):
    print(f"  {i}: {class_names[i]}")

In [ ]:
# Load the trained model
if MODEL_PATH.exists():
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully!")
    print(f"\n📐 Model Architecture:")
    model.summary()
else:
    print("❌ Model not found. Please train the model first using train_mobilenet.py")

## 🔬 Test on Real Medicinal Plant Images

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):
    """Preprocess image for model prediction"""
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_plant(img_path, model, class_names, top_k=5):
    """Predict plant species from image"""
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array, verbose=0)[0]
    
    # Get top K predictions
    top_indices = np.argsort(predictions)[::-1][:top_k]
    top_predictions = [
        {
            'class': class_names[idx],
            'confidence': float(predictions[idx])
        }
        for idx in top_indices
    ]
    
    return top_predictions

print("✅ Prediction functions ready!")

In [ ]:
# Select random test images from different classes
test_images = []
test_classes = ['Tulsi', 'Aloevera', 'Neem', 'Mint', 'Betel']  # Sample classes

for class_name in test_classes:
    class_dir = DATASET_PATH / class_name
    if class_dir.exists():
        images = list(class_dir.glob('*.jpg'))[:1]  # Take first image
        if images:
            test_images.append({
                'path': images[0],
                'true_class': class_name
            })

print(f"📸 Selected {len(test_images)} test images")

## 🎯 Visualize Predictions

In [ ]:
# Test and visualize predictions
fig, axes = plt.subplots(len(test_images), 2, figsize=(15, 5 * len(test_images)))
if len(test_images) == 1:
    axes = axes.reshape(1, -1)

for idx, test_img in enumerate(test_images):
    # Load and display image
    img = Image.open(test_img['path'])
    axes[idx, 0].imshow(img)
    axes[idx, 0].set_title(f"True Class: {test_img['true_class']}", fontsize=14, fontweight='bold')
    axes[idx, 0].axis('off')
    
    # Get predictions
    predictions = predict_plant(test_img['path'], model, class_names, top_k=5)
    
    # Plot confidence bars
    classes = [p['class'] for p in predictions]
    confidences = [p['confidence'] * 100 for p in predictions]
    colors = ['#16a34a' if i == 0 else '#94a3b8' for i in range(len(classes))]
    
    axes[idx, 1].barh(classes, confidences, color=colors)
    axes[idx, 1].set_xlabel('Confidence (%)', fontsize=12)
    axes[idx, 1].set_title(f"Top 5 Predictions", fontsize=14, fontweight='bold')
    axes[idx, 1].set_xlim(0, 100)
    
    # Add confidence values on bars
    for i, (cls, conf) in enumerate(zip(classes, confidences)):
        axes[idx, 1].text(conf + 2, i, f"{conf:.1f}%", va='center', fontsize=10)
    
    # Highlight if prediction is correct
    is_correct = predictions[0]['class'] == test_img['true_class']
    result_text = "✅ CORRECT" if is_correct else "❌ INCORRECT"
    result_color = 'green' if is_correct else 'red'
    axes[idx, 1].text(0.5, -0.15, result_text, transform=axes[idx, 1].transAxes,
                     fontsize=14, fontweight='bold', color=result_color, ha='center')

plt.tight_layout()
plt.savefig('model_predictions_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Prediction Results Saved!")

## 📈 Model Performance Analysis

In [ ]:
# Calculate accuracy on test images
correct = 0
total = len(test_images)

print("\n🎯 Detailed Prediction Analysis:\n")
print("=" * 80)

for test_img in test_images:
    predictions = predict_plant(test_img['path'], model, class_names, top_k=3)
    predicted_class = predictions[0]['class']
    confidence = predictions[0]['confidence'] * 100
    is_correct = predicted_class == test_img['true_class']
    
    if is_correct:
        correct += 1
    
    status = "✅ CORRECT" if is_correct else "❌ WRONG"
    print(f"\n{status}")
    print(f"  True Class:      {test_img['true_class']}")
    print(f"  Predicted:       {predicted_class}")
    print(f"  Confidence:      {confidence:.2f}%")
    print(f"  Top 3 Predictions:")
    for i, pred in enumerate(predictions[:3], 1):
        print(f"    {i}. {pred['class']}: {pred['confidence']*100:.2f}%")
    print("-" * 80)

accuracy = (correct / total) * 100 if total > 0 else 0
print(f"\n\n🏆 Overall Test Accuracy: {accuracy:.2f}% ({correct}/{total})")

## 🔥 Confidence Distribution Analysis

In [ ]:
# Analyze confidence distribution
all_confidences = []
for test_img in test_images:
    predictions = predict_plant(test_img['path'], model, class_names, top_k=1)
    all_confidences.append(predictions[0]['confidence'] * 100)

# Plot confidence distribution
plt.figure(figsize=(10, 6))
plt.hist(all_confidences, bins=10, color='#16a34a', alpha=0.7, edgecolor='black')
plt.axvline(np.mean(all_confidences), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(all_confidences):.1f}%')
plt.xlabel('Confidence (%)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Model Confidence Distribution', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.savefig('confidence_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Confidence Statistics:")
print(f"  Mean Confidence:   {np.mean(all_confidences):.2f}%")
print(f"  Median Confidence: {np.median(all_confidences):.2f}%")
print(f"  Min Confidence:    {np.min(all_confidences):.2f}%")
print(f"  Max Confidence:    {np.max(all_confidences):.2f}%")

## 🎓 Model Intelligence Summary

### Key Findings:
- **Architecture**: MobileNetV2 (Lightweight & Efficient)
- **Classes**: 80 Medicinal Plant Species
- **Input Size**: 224x224 RGB Images
- **Deployment**: Production-ready with ONNX optimization

### Next Steps for Maximum Power:
1. **Train EfficientNetV2**: Run `train_enhanced.py` for higher accuracy
2. **Enable Gemini Expert**: Add API key to `.env` for AI verification
3. **Expand Dataset**: Add more images per class for better generalization

---
**Developed by Group G9** | Dr. DY Patil College of Eng. and Innovation